 # **Creating a tool to receive instant stock prices with yfinance**

### **Step 1 : Install the required libraries**

In [ ]:
!pip install yfinance langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 3.9 MB/s eta 0:00:00


### **Step 2: Write the function needed to receive the information**

In [ ]:
import yfinance as yf
from langchain.tools import Tool

def get_stock_price(ticker: str) -> str:
    data = yf.Ticker(ticker).history(period="1d")
    price = data["Close"].iloc[-1]
    return f"The current price of {ticker.upper()} is ${price:.2f}"

In [ ]:
get_stock_price_tool = Tool(
    name="get_stock_price",
    func=get_stock_price,
    description="Get the current stock price for a given ticker symbol (e.g., AAPL, GOOGL)."
)

tools = [get_stock_price_tool]

**Now test our function**

In [ ]:
print(get_stock_price("AAPL"))

The current price of AAPL is $226.01


### **Step 3: Convert Python function to langchain tool and build agent**

#### **create our LLM**

In [ ]:
import os
from google.colab import userdata

openai_api_key = userdata.get('Openai_api_key')
os.environ["OPENAI_API_KEY"] = openai_api_key


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

#### **Now it's time to create our Agent**

In [ ]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

#### **Now we can get the Price of AAPL & GOOGL**

In [ ]:
response = agent.run("What is the current price of AAPL stock?")
print(response)

/tmp/ipython-input-358769495.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run("What is the current price of AAPL stock?")




> Entering new AgentExecutor chain...
I need to find the current stock price for AAPL. 
Action: get_stock_price
Action Input: "AAPL"
Observation: The current price of AAPL is $226.01
Thought:I now know the final answer.  
Final Answer: The current price of AAPL stock is $226.01.

> Finished chain.
The current price of AAPL stock is $226.01.


In [ ]:
response = agent.run("What is the current price of GOOGL stock?")
print(response)



> Entering new AgentExecutor chain...
I need to find the current stock price for GOOGL. 
Action: get_stock_price
Action Input: "GOOGL"
Observation: The current price of GOOGL is $199.32
Thought:I now know the final answer.  
Final Answer: The current price of GOOGL stock is $199.32.

> Finished chain.
The current price of GOOGL stock is $199.32.
